In [71]:
import time
import random
import csv
from typing import Tuple, Dict, Optional

import requests
from bs4 import BeautifulSoup

In [20]:
import lxml

In [35]:
url = "https://cbr.ru"

In [36]:
response = requests.get(url=url+"/faq/")

In [4]:
response.text

'\r\n<!DOCTYPE html>\r\n<html>\r\n<head>\r\n    \r\n\r\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\r\n<meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\r\n<meta name="format-detection" content="telephone=no" />\r\n<meta name="zoom:lang" content="ru" />\r\n    <meta name="zoom:last-modified" content="Fri, 26 Jan 2024 06:39:25 GMT" />\r\n<title>Вопросы и ответы | Банк России</title>\r\n\r\n    <meta name="keywords" content="часто задаваемые вопросы" />\r\n\r\n\r\n    <meta property="og:image" content="/common/images/share-1.jpg" />\r\n\r\n    \r\n\r\n\r\n            <link rel="stylesheet" type="text/css" href="/common/libs/jquery-ui/jquery-ui.min.css?v=v478607061" media="all">\r\n\r\n            <!--[if IE 9]><link rel="stylesheet" type="text/css" href="/common/style/main-ie9.css?v=v2803087654" media="all"><![endif]-->\r\n\r\n            <!--[if gt IE 9]><!-->

In [38]:
soup = BeautifulSoup(response.text)

In [23]:
soup


<!DOCTYPE html>

<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="telephone=no" name="format-detection">
<meta content="ru" name="zoom:lang">
<meta content="Fri, 26 Jan 2024 06:39:25 GMT" name="zoom:last-modified"/>
<title>Вопросы и ответы | Банк России</title>
<meta content="часто задаваемые вопросы" name="keywords"/>
<meta content="/common/images/share-1.jpg" property="og:image"/>
<link href="/common/libs/jquery-ui/jquery-ui.min.css?v=v478607061" media="all" rel="stylesheet" type="text/css"/>
<!--[if IE 9]><link rel="stylesheet" type="text/css" href="/common/style/main-ie9.css?v=v2803087654" media="all"><![endif]-->
<!--[if gt IE 9]><!--><link href="/common/style/main.min.css?v=v2295644851" media="all" rel="stylesheet" type="text/css"/><!--<![endif]-->
<script src="/common/libs/jquery-

In [24]:
rubrics = soup.find(class_="rubric_title")

In [25]:
rubrics = soup.find_all(class_="rubric_title")
rubrics

[<a class="rubric_title" href="/faq/w_fin_sector/">Актуальные вопросы работы финансовой системы</a>,
 <a class="rubric_title" href="/faq/dkp/">Денежно-кредитная политика</a>,
 <a class="rubric_title" href="/faq/ndo/">Наличное денежное обращение</a>,
 <a class="rubric_title" href="/faq/nps/">Национальная платежная система</a>,
 <a class="rubric_title" href="/faq/bank_s/">Банковский сектор</a>,
 <a class="rubric_title" href="/faq/pens_k_inv/">Пенсионные фонды и коллективные инвестиции</a>,
 <a class="rubric_title" href="/faq/insh/">Страхование</a>,
 <a class="rubric_title" href="/faq/rcb/">Рынок ценных бумаг</a>,
 <a class="rubric_title" href="/faq/finm_infrastructure/">Инфраструктура финансового рынка</a>,
 <a class="rubric_title" href="/faq/microfin/">Микрофинансирование</a>,
 <a class="rubric_title" href="/faq/credit_h/">Кредитные истории</a>,
 <a class="rubric_title" href="/faq/fintekh/">Развитие финансовых технологий</a>,
 <a class="rubric_title" href="/faq/pnp/">Противодействие нед

In [76]:
def extract_questions(dropdown_questions: Optional[Tuple], rubric_title: str, theme_name: str = "") -> Tuple[Dict]:
    items = []
    for dropdown_question in dropdown_questions:
        try:
            question = dropdown_question.find("h3") if dropdown_question.find("h3") else dropdown_question.find("h2")
            question = question.get_text().strip().replace("\xa0", " ")
            answer = dropdown_question.find("div", class_="additional-text-block").get_text().strip().replace("\xa0", " ")
            
            items.append({"title": rubric_title,"theme": theme_name, "question": question, "answer": answer})
        except Exception:
            print(f"Failed to parse question. Rubric: {rubric_title} \nTheme: {theme_name} \nDropdown_question: \n{dropdown_question}")

    return items

In [77]:
data = []
for rubric in rubrics:
    rubric_title = rubric.get_text().replace("\xa0", " ")
    rubric_url = url + rubric.get("href")
    print(rubric_title, rubric_url)

    res = requests.get(rubric_url)
    rubric_soup = BeautifulSoup(res.text)
    categories = rubric_soup.find_all("div", class_="dropdown dropdown_container")

    if not categories:
        dropdown_questions = rubric_soup.find_all("div", class_="dropdown question")
        items = extract_questions(
            dropdown_questions=dropdown_questions, rubric_title=rubric_title
        )
        data.extend(items)
        continue

    for category in categories:
        theme = category.find("div", class_="dropdown_title referenceable")
        if theme:
            theme_name = theme.find("h2").get_text().replace("\xa0", " ")
            dropdown_questions = category.find_all("div", class_="dropdown question")
            items = extract_questions(
                dropdown_questions=dropdown_questions, rubric_title=rubric_title, theme_name=theme_name
            )
            data.extend(items)
        else:
            time.sleep(random.randint(1, 2))

            theme = category.find("div", class_="dropdown_title-link").find("a", class_="dropdown_link referenceable")
            theme_url = url + theme.get("href")
            theme_name = theme.get_text().replace("\xa0", " ")

            theme_response = requests.get(theme_url)
            theme_soup = BeautifulSoup(theme_response.text)

            dropdown_questions = theme_soup.find_all("div", class_="dropdown question")
            items = extract_questions(
                dropdown_questions=dropdown_questions, rubric_title=rubric_title, theme_name=theme_name
            )
            data.extend(items)


        # print(theme_name)

        # answer = category.find()
        

Актуальные вопросы работы финансовой системы https://cbr.ru/faq/w_fin_sector/
Денежно-кредитная политика https://cbr.ru/faq/dkp/
Наличное денежное обращение https://cbr.ru/faq/ndo/
Национальная платежная система https://cbr.ru/faq/nps/
Банковский сектор https://cbr.ru/faq/bank_s/
Пенсионные фонды и коллективные инвестиции https://cbr.ru/faq/pens_k_inv/
Страхование https://cbr.ru/faq/insh/
Рынок ценных бумаг https://cbr.ru/faq/rcb/
Инфраструктура финансового рынка https://cbr.ru/faq/finm_infrastructure/
Микрофинансирование https://cbr.ru/faq/microfin/
Кредитные истории https://cbr.ru/faq/credit_h/
Развитие финансовых технологий https://cbr.ru/faq/fintekh/
Противодействие недобросовестным практикам https://cbr.ru/faq/pnp/
Платформа «Знай своего клиента»  https://cbr.ru/faq/client/
О Банке России https://cbr.ru/faqnew/br/
Информационная безопасность https://cbr.ru/faq/information_security/
Жалобы в Банк России  https://cbr.ru/faq/zhaloby-v-bank-rossii-/
Удостоверяющий центр Банка России h

In [78]:
data[:3]

[{'title': 'Актуальные вопросы работы финансовой системы',
  'theme': 'Меры поддержки мобилизованных и участников СВО',
  'question': 'Могу ли я, как участник СВО, переводить деньги без комиссии?',
  'answer': 'Банк России рекомендовал банкам снизить или отменить комиссии за переводы со счета участника СВО членам его семьи и за снятие наличных со счетов участников СВО и членов их семей.\nБанки самостоятельно устанавливают порядок реализации этих мер поддержки. Согласно рекомендациям Банка России, военнослужащий должен иметь возможность в доступной для него форме или через представителя сообщить банку о наличии оснований для сниженных комиссий и о том, на кого из членов семьи они должны распространяться.'},
 {'title': 'Актуальные вопросы работы финансовой системы',
  'theme': 'Меры поддержки мобилизованных и участников СВО',
  'question': 'Что такое кредитные каникулы для мобилизованных и участников СВО? По каким кредитам мобилизованные и другие участники СВО могут получить кредитные ка

In [79]:
len(data)

637

In [84]:
csv_file_path = "../data/questions.csv"
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames=["title", "theme", "question", "answer"])
    
    csv_writer.writeheader()
    csv_writer.writerows(data)

print(f"Данные успешно сохранены в файл: {csv_file_path}")

Данные успешно сохранены в файл: ../data/questions.csv


In [ ]:
data = [{"theme": "", "question": "", "answer": ""}, ...]
data = {"some_theme": {"question": "", "answer": ""}}